In [114]:
import os

In [115]:
%pwd

'C:\\Users\\abdus samad\\Desktop\\Churn_ML\\Churn_ML_Project'

In [116]:
os.chdir("C:/Users/abdus samad/Desktop/Churn_ML/Churn_ML_Project")
%pwd

'C:\\Users\\abdus samad\\Desktop\\Churn_ML\\Churn_ML_Project'

In [117]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: str
    learning_rate: float
    max_depth: int
    n_estimators: int
    scale_pos_weight: float
    subsample: float
    colsample_bytree: float
    random_state: int

In [118]:
from src.Churn_Predictor.constants import *
from src.Churn_Predictor.utils.common import read_yaml, create_directories

In [123]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        model_trainer_config = self.config.model_trainer
        model_trainer_params = self.params.XGBBoost
        target_column = list(self.schema.TARGET_COLUMN.keys())[0]
        create_directories([model_trainer_config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=model_trainer_config.root_dir,
            train_data_path=model_trainer_config.train_data_path,
            test_data_path=model_trainer_config.test_data_path,
            model_name=model_trainer_config.model_name,
            target_column=target_column,
            learning_rate=model_trainer_params.learning_rate,
            max_depth=model_trainer_params.max_depth,
            n_estimators=model_trainer_params.n_estimators,
            scale_pos_weight=model_trainer_params.scale_pos_weight,
            subsample=model_trainer_params.subsample,
            colsample_bytree=model_trainer_params.colsample_bytree,
            random_state=model_trainer_params.random_state
        )
        
        return model_trainer_config

In [124]:
import pandas as pd
from xgboost import XGBClassifier
from src.Churn_Predictor import logger
import joblib

In [125]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train_model(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop(columns=[self.config.target_column])
        y_train = train_data[self.config.target_column]
        X_test = test_data.drop(columns=[self.config.target_column])
        y_test = test_data[self.config.target_column]

        xgb = XGBClassifier(
            learning_rate=self.config.learning_rate,
            max_depth=self.config.max_depth,
            n_estimators=self.config.n_estimators,
            scale_pos_weight=self.config.scale_pos_weight,
            subsample=self.config.subsample,
            colsample_bytree=self.config.colsample_bytree,
            random_state=self.config.random_state
        )
        xgb.fit(X_train, y_train)
        logger.info("Model training completed.")
    
        joblib.dump(xgb, os.path.join(self.config.root_dir, self.config.model_name))

In [126]:
try:
    config = ConfigurationManager()
    modeltrainer_config = config.get_model_trainer_config()
    modeltrainer = ModelTrainer(config=modeltrainer_config)
    modeltrainer.train_model()
except Exception as e:
    logger.exception(f"Error occurred: {e}")

[2026-02-13 19:35:52,605: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-13 19:35:52,608: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-13 19:35:52,614: INFO: common: yaml file: schema.yaml loaded successfully]
[2026-02-13 19:35:52,616: INFO: common: created directory at: artifacts]
[2026-02-13 19:35:52,617: INFO: common: created directory at: artifacts/model_trainer]
[2026-02-13 19:35:52,781: INFO: 4047964950: Model training completed.]
